In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
import seaborn as sns
from math import floor, ceil
from pylab import rcParams

# Loading Keras packeges:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Reshape, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import SGD

import os
print(os.listdir("input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'train.csv']


In [3]:
# Loading data:
train_df = pd.read_csv('input/train.csv')
test_df = pd.read_csv('input/test.csv')

train_df.shape, test_df.shape

((42000, 785), (28000, 784))

In [4]:
# Assining to X_train+CV and X_test, and seperate train to labels and data:
X_train_CV = train_df.drop("label", axis=1)
Y_train_CV_raw = train_df["label"]
X_test  = test_df
X_train_CV.shape, Y_train_CV_raw.shape, X_test.shape

((42000, 784), (42000,), (28000, 784))

In [5]:
# Converting Y_train to 0,1 format:
Y_train_CV = pd.DataFrame()

for i in range (0,10):
    Y_train_CV.loc[:, i] = pd.Series(((Y_train_CV_raw == i) *1)) 

X_train_CV.head(10)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Final input/output shapes:
X_train_CV.shape, Y_train_CV.shape, X_test.shape

((42000, 784), (42000, 10), (28000, 784))

In [7]:
# Deviding between train and CV sets:
train_cut = floor(X_train_CV.shape[0] * 0.85)
X_train = X_train_CV.iloc[0:train_cut].values
Y_train = Y_train_CV.iloc[0:train_cut].values
X_CV = X_train_CV.iloc[train_cut:].values
Y_CV = Y_train_CV.iloc[train_cut:].values

X_train.shape, Y_train.shape, X_CV.shape, Y_CV.shape

((35700, 784), (35700, 10), (6300, 784), (6300, 10))

In [ ]:
# Making sure tensorflow works with GPU:

#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))


In [8]:
# Building CNN model:

model = Sequential()

# Reshape input layer:
model.add(Reshape((28, 28, 1), input_shape=(X_train.shape[1],)))
model.add(Conv2D(4, (2, 2), activation='relu', ))
model.add(Conv2D(8, (2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(16, (2, 2), activation='relu'))
model.add(Conv2D(32, (2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# Train and fit the model:
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=4, verbose=1, batch_size=32)

scores = model.evaluate(X_train, Y_train)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/4
35700/35700 [==============================] - 19s 543us/step - loss: 1.8567 - acc: 0.7480
Epoch 2/4
35700/35700 [==============================] - 14s 393us/step - loss: 0.2380 - acc: 0.9265
Epoch 3/4
35700/35700 [==============================] - 14s 389us/step - loss: 0.1701 - acc: 0.9476
Epoch 4/4
35700/35700 [==============================] - 3s 94us/step

acc: 98.44%


In [9]:
# Evaluate model on CV set:
scores = model.evaluate(X_CV, Y_CV)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

6300/6300 [==============================] - 1s 97us/step

acc: 98.06%


In [10]:
# Predicting on test set:
Y_pred_raw = pd.DataFrame(model.predict(X_test, verbose=1, steps=None)).idxmax(axis=1)

28000/28000 [==============================] - 2s 79us/step


In [11]:
# Creating the output with the ImageId:

ImageId = pd.Series(np.arange(1, Y_pred_raw.size+1))
Y_pred = pd.DataFrame(Y_pred_raw, columns={'Label'})
Y_pred['ImageId'] = pd.Series(ImageId, index=Y_pred.index)


In [12]:
# Outputing the CSV file:
submission = pd.DataFrame({
        "ImageId": Y_pred["ImageId"],
        "Label": Y_pred["Label"]
    })

submission.head(10)

submission.to_csv("submission.csv", index=False)